# 

In [35]:
from app.database import *
from app.models import *
from app.database.database import *

In [37]:
async_engine.url

postgresql+asyncpg://A2SV Bouncer:***@34.165.255.238/Attendance

In [153]:
start_date = datetime(month=9, day=1, year=2023).date()
end_date = datetime.now().date()
one_day = timedelta(days=1)

In [154]:
current_date = start_date
columns = ['Name', 'Project']
while(current_date <= end_date):
    columns.append(str(current_date))
    current_date  += one_day

In [155]:
users : list[User] = []
async with get_db() as session:
    query = select(User)\
        .options(selectinload(User.timesheets))\
        .options(selectinload(User.project))\
        .order_by(User.full_name)
    print(query)
    res = await session.execute(query)
    res = res.scalars().all()
    users = list(res)

SELECT "user".slack_id, "user".full_name, "user".role, "user".employement_status, "user".daily_plan_channel, "user".headsup_channel, "user".check_in_check_out_channel, "user".is_admin, "user".profile_url, "user".timezone, "user".project_id 
FROM "user" ORDER BY "user".full_name


In [156]:
len(users)

218

In [157]:
import pandas as pd


In [164]:
df = pd.DataFrame(columns=columns)

In [165]:
for user in users:
    row = [0.0] * len(columns)
    row[0] = user.full_name
    row[1] = user.project.name
    df.loc[len(df)] = row

In [166]:
for idx,user in enumerate(users):
    for time_sheet in user.timesheets:
        if not time_sheet.check_out_time:
            continue
        dt = time_sheet.check_in_time.date()
        if dt < start_date or dt > end_date:
            continue
        tm = str(time_sheet.check_in_time.date())
        df.at[idx,tm] += time_sheet.elapsed_time/3600

In [159]:
users[0].project.name

'G4 second year education'

In [170]:
df.to_csv('september-jan19.csv')